In [1]:
!pip install elasticsearch==8.11.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 412.6/412.6 kB 3.1 MB/s eta 0:00:0000:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 2.1 MB/s eta 0:00:00


In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .master("spark://spark-master:7077") \
    .appName("JupyterSparkApp") \
    .getOrCreate()

In [3]:
from elasticsearch import Elasticsearch

# Elasticsearch 클라이언트 초기화
es = Elasticsearch(
    ["http://elasticsearch:9200"],
    headers={
        "Accept": "application/vnd.elasticsearch+json; compatible-with=8",
        "Content-Type": "application/json"
    }
)

In [4]:
sample_data = [
    ("Alice", 25, "Engineer", "Seoul"),
    ("Bob", 30, "Designer", "Busan"), 
    ("Charlie", 35, "Manager", "Daegu"),
    ("Diana", 28, "Analyst", "Seoul"),
    ("Eve", 32, "Developer", "Incheon")
]

In [5]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

schema = StructType([
    StructField("name", StringType(), True),
    StructField("age", IntegerType(), True),
    StructField("job", StringType(), True),
    StructField("city", StringType(), True)
])

In [6]:
df = spark.createDataFrame(sample_data, schema)

In [7]:
!pip show pyspark

Name: pyspark
Version: 3.5.0
Summary: Apache Spark Python API
Home-page: https://github.com/apache/spark/tree/master/python
Author: Spark Developers
Author-email: dev@spark.apache.org
License: http://www.apache.org/licenses/LICENSE-2.0
Location: /usr/local/spark/python
Requires: py4j
Required-by: 


In [9]:
doc = {
        'name': 'Frank',
        'age': 45,
        'job': 'CEO',
        'city': 'Seoul',
        'timestamp': '2024-01-01T12:00:00'
}
    
# 문서 추가
response = es.index(index="employees", id=6, body=doc)
print(f"문서 추가 결과: {response['result']}")

Traceback (most recent call last):
  File "/opt/conda/lib/python3.11/site-packages/urllib3/connection.py", line 203, in _new_conn
    sock = connection.create_connection(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/conda/lib/python3.11/site-packages/urllib3/util/connection.py", line 85, in create_connection
    raise err
  File "/opt/conda/lib/python3.11/site-packages/urllib3/util/connection.py", line 73, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [Errno 111] Connection refused

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/opt/conda/lib/python3.11/site-packages/elastic_transport/_node/_http_urllib3.py", line 167, in perform_request
    response = self.pool.urlopen(
               ^^^^^^^^^^^^^^^^^^
  File "/opt/conda/lib/python3.11/site-packages/urllib3/connectionpool.py", line 845, in urlopen
    retries = retries.increment(
              ^^^^^^^^^^^^^^^^^^
  File "/opt/conda/lib/pyth

ConnectionError: Connection error caused by: ConnectionError(Connection error caused by: NewConnectionError(<urllib3.connection.HTTPConnection object at 0x7fb92905c1d0>: Failed to establish a new connection: [Errno 111] Connection refused))

In [ ]:
# 1. 인덱스 리스트 조회
print("=== 인덱스 리스트 ===")
indices = es.cat.indices(format="json")
for index in indices:
    index_name = index['index']
    doc_count = index['docs.count']
    print(f"인덱스: {index_name}, 문서 수: {doc_count}")

In [ ]:
# 2. 특정 인덱스의 문서 내용 조회 (예: 'employees' 인덱스)
print("\n=== 'employees' 인덱스 문서 내용 ===")
search_result = es.search(
    index="employees",
    body={
        "query": {
            "match_all": {}  # 모든 문서 조회
        },
        "size": 10  # 최대 10개 문서 반환
    }
)

In [ ]:
# 검색 결과 출력
hits = search_result['hits']['hits']
if hits:
    for hit in hits:
        print(f"문서 ID: {hit['_id']}, 내용: {hit['_source']}")
else:
    print("인덱스 'employees'에 문서가 없습니다.")

In [11]:
spark.stop()

In [12]:
df.show()

Py4JJavaError: An error occurred while calling o39.showString.
: java.lang.IllegalStateException: SparkContext has been shutdown
	at org.apache.spark.SparkContext.runJob(SparkContext.scala:2390)
	at org.apache.spark.SparkContext.runJob(SparkContext.scala:2419)
	at org.apache.spark.SparkContext.runJob(SparkContext.scala:2438)
	at org.apache.spark.sql.execution.SparkPlan.executeTake(SparkPlan.scala:530)
	at org.apache.spark.sql.execution.SparkPlan.executeTake(SparkPlan.scala:483)
	at org.apache.spark.sql.execution.CollectLimitExec.executeCollect(limit.scala:61)
	at org.apache.spark.sql.Dataset.collectFromPlan(Dataset.scala:4344)
	at org.apache.spark.sql.Dataset.$anonfun$head$1(Dataset.scala:3326)
	at org.apache.spark.sql.Dataset.$anonfun$withAction$2(Dataset.scala:4334)
	at org.apache.spark.sql.execution.QueryExecution$.withInternalError(QueryExecution.scala:546)
	at org.apache.spark.sql.Dataset.$anonfun$withAction$1(Dataset.scala:4332)
	at org.apache.spark.sql.execution.SQLExecution$.$a

In [26]:
!pip uninstall pyspark

Found existing installation: pyspark 3.5.0
Can't uninstall 'pyspark'. No files were found to uninstall.


In [27]:
!pip install pyspark==3.5.6

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.4/317.4 MB 1.3 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.5/200.5 kB 9.2 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.5.6-py2.py3-none-any.whl size=317895797 sha256=91f6a655deded1e1e9cf8699920f04a336e6d6824ed8597565568b773499b489
  Stored in directory: /home/jovyan/.cache/pip/wheels/fc/97/82/2e7d88fca39c390b8f0745ff0719516a806a697e5e095ada48
Successfully built pyspark
  Attempting uninstall: pyspark
    Found existing installation: pyspark 3.5.0
    Can't uninstall 'pyspark'. No files were found to uninstall.


In [28]:
!pip show pyspark

Name: pyspark
Version: 3.5.0
Summary: Apache Spark Python API
Home-page: https://github.com/apache/spark/tree/master/python
Author: Spark Developers
Author-email: dev@spark.apache.org
License: http://www.apache.org/licenses/LICENSE-2.0
Location: /usr/local/spark/python
Requires: py4j
Required-by: 
